# Linear Quantization - Quanto

[Quanto code](https://colab.research.google.com/drive/16CXfVmtdQvciSh9BopZUDYcmXCDpvgrT?usp=sharing#scrollTo=fdi4O-CHgEZG)

In [ ]:
!pip install -U -q git+https://github.com/huggingface/transformers.git
!pip install -U -q quanto
!pip install -U -q accelerate
!pip install -q datasets
!pip install einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 13.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, QuantoConfig
import torch

import torch
from torch.utils.data import TensorDataset,DataLoader, RandomSampler, SequentialSampler
import pandas as pd
from tqdm import tqdm

In [ ]:
model_id = "internlm/internlm2-7b"

In [ ]:
quantization_config = QuantoConfig(weights="int8")

Internlm_quanto_model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32, quantization_config=quantization_config, device_map="auto")
Internlm_quanto_tokenizer = AutoTokenizer.from_pretrained(model_id)

The repository for internlm/internlm2-7b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/internlm/internlm2-7b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
The repository for internlm/internlm2-7b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/internlm/internlm2-7b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


pytorch_model.bin.index.json:   0%|          | 0.00/18.2k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.51G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenization_internlm2_fast.py:   0%|          | 0.00/7.80k [00:00<?, ?B/s]

tokenization_internlm2.py:   0%|          | 0.00/8.81k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2-7b:
- tokenization_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2-7b:
- tokenization_internlm2_fast.py
- tokenization_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


./tokenizer.model:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

**FOR INT4**

In [ ]:
# quantization_config_int4 = QuantoConfig(weights="int4")

# Internlm_quanto_modelg_int4 = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32, quantization_config=quantization_config, device_map="auto")
# Internlm_quanto_tokenizerg_int4 = AutoTokenizer.from_pretrained(model_id)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# internlm7b_tokenizer.to(device) #'InternLM2TokenizerFast' object has no attribute 'to'
Internlm_quanto_model.to(device)


InternLM2ForCausalLM(
  (model): InternLM2Model(
    (tok_embeddings): Embedding(92544, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x InternLM2DecoderLayer(
        (attention): InternLM2Attention(
          (wqkv): QLinear(in_features=4096, out_features=6144, bias=False)
          (wo): QLinear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): InternLM2RotaryEmbedding()
        )
        (feed_forward): InternLM2MLP(
          (w1): QLinear(in_features=4096, out_features=14336, bias=False)
          (w3): QLinear(in_features=4096, out_features=14336, bias=False)
          (w2): QLinear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (attention_norm): InternLM2RMSNorm()
        (ffn_norm): InternLM2RMSNorm()
      )
    )
    (norm): InternLM2RMSNorm()
  )
  (output): Linear(in_features=4096, out_features=92544, bias=False)
)

In [ ]:
prompt = ["Predict the sentiment of the input. Answer either {positive/negative/neutral}. Input: I hate you. Answer:"]
inputs = Internlm_quanto_tokenizer(prompt, return_tensors="pt")

inputs.to(device)
# for k,v in inputs.items():
#     inputs[k] = v.cuda()
# gen_kwargs = {"max_length": 128, "top_p": 0.8, "temperature": 0.8, "do_sample": True, "repetition_penalty": 1.0}
output = Internlm_quanto_model.generate(**inputs,max_length=100) #**gen_kwargs)
output = Internlm_quanto_tokenizer.decode(output[0].tolist(), skip_special_tokens=True)
print(output)

Predict the sentiment of the input. Answer either {positive/negative/neutral}. Input: I hate you. Answer: negative.


# Imbalanced dataset - Financial phrasebank

In [ ]:
from datasets import load_dataset

fpb_dataset = load_dataset("financial_phrasebank",'sentences_allagree')

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

**Format the dataset as a prompt**

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# tokenizer.to(device)

In [ ]:
labels = "positive/negative/neutral"
p=[]
for i in range(len(fpb_dataset['train']['sentence'])):
  p.append(f"Predict the sentiment of the input. Answer either {labels}. Input: {fpb_dataset['train']['sentence'][i]}. Answer:")

fpb_dataset['train'] = fpb_dataset['train'].add_column("prompt_2", p)

In [ ]:
fpb_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'prompt_2'],
        num_rows: 2264
    })
})

In [ ]:
# tokenized_set_1 = flant5_tokenizer(prompt, return_tensors='pt', padding=True,max_length=512)

#---------------------------------------------------------

tokenized_set_quanto = Internlm_quanto_tokenizer.batch_encode_plus(
    fpb_dataset['train']['prompt_2'], #prompt
    return_tensors='pt',
    add_special_tokens=True,
    return_attention_mask=True,
    # pad_to_max_length=True,  Deprecated
    padding=True,
    truncation=True,
    max_length=175 # the maximum lenght observed in the headlines, as can be seen in the above distribution
)

In [ ]:
input_ids_val = tokenized_set_quanto['input_ids']
attention_masks_val = tokenized_set_quanto['attention_mask']

dataset_val = TensorDataset(input_ids_val, attention_masks_val)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=5)

In [ ]:
import time, torch
predictions_quanto = []
start_time = time.time()
Internlm_quanto_model.eval()

for batch in dataloader_validation:

  batch = tuple(b.to(device) for b in batch)

  inputs = {'input_ids':batch[0],
                  'attention_mask': batch[1],
                 }

  with torch.no_grad():
    outputs = Internlm_quanto_model.generate(**inputs, max_length=200)
    # outputs = flant5_model(**inputs)

  res_sentences = [Internlm_quanto_tokenizer.decode(i) for i in outputs]
  # out_text = [o.split("Answer:")[1] for o in res_sentences]

  # # show results
  # for gen_text in out_text:
  #     print(gen_text)

  end_time = time.time()
  execution_time = end_time - start_time

  predictions_quanto.append(res_sentences)
# print("Result:", res_sentences)
print("Execution time:", execution_time, "seconds")

Execution time: 2174.4022500514984 seconds


**Extract response from models output**

In [ ]:
final_sent_answer_quanto=[]
for batch in predictions_quanto:
  for sent_output in batch:
    final_sent_answer_quanto.append(sent_output.split('Answer:')[1].split('<')[0])

In [ ]:
for index,res in enumerate(final_sent_answer_quanto):
  final_sent_answer_quanto[index] = res.replace(" ","")

In [ ]:
len(final_sent_answer_quanto)

2264

In [ ]:
# prompt: convert list to dataframe with column called 'response'

import pandas as pd
df_res = pd.DataFrame(final_sent_answer_quanto, columns=['response'])


In [ ]:
df_res['label'] = [1] *2264

In [ ]:
df_res.head(5)

,response,label
0,neutral,1
1,Positive,2
2,positive,2
3,positive,2
4,positive,2


In [ ]:
for index,res in enumerate(df_res['response']):
  # print(index)
  if df_res['response'][index].lower() == 'positive':
    df_res['label'][index] = 2
  elif df_res['response'][index].lower() == 'negative':
    df_res['label'][index] = 0
  elif df_res['response'][index].lower() == 'neutral':
    df_res['label'][index] = 1


In [ ]:
import pandas as pd
df_res.to_csv('final_sent_answer_Internlm-7b_quanto.csv')

## **Evaluate Model**

In [ ]:
# prompt: code to convert dataset object fpb_dataset to pandas dataframe object

fpb_dataset_pandas = fpb_dataset['train'].to_pandas()
# Compare the values in the two columns
comparison = fpb_dataset_pandas["label"] == df_res["label"]

# Calculate the accuracy
accuracy = comparison.sum() / len(comparison)

# Print the accuracy
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.86


In [ ]:
# prompt: code to calculate precision

from sklearn.metrics import precision_score

y_true = fpb_dataset['train']["label"]
y_pred = df_res["label"]

precision = precision_score(y_true, y_pred, average='macro')
print("Macro-averaged precision:", precision)

Macro-averaged precision: 0.9029295212351881


In [ ]:
# prompt: code to calculate f1 score

from sklearn.metrics import f1_score

# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='macro')

# Print F1 score
print("Macro-averaged F1 score:", f1)


Macro-averaged F1 score: 0.8378365389333408


In [ ]:
# prompt: code to calculate recall

from sklearn.metrics import recall_score

# Calculate recall score
recall = recall_score(y_true, y_pred, average='macro')

# Print recall score
print("Macro-averaged recall:", recall)


Macro-averaged recall: 0.7967666197926312


# Balanced dataset - Financial news headlines

[Dataset link](https://huggingface.co/datasets/steve1989/financial_news_headlines)

In [ ]:
from datasets import load_dataset

financial_dataset = load_dataset("steve1989/financial_news_headlines", split="test")

In [ ]:
financial_dataset

Dataset({
    features: ['Headlines', 'sentiment_label', '__index_level_0__'],
    num_rows: 2793
})

Add **label** column to dataset

In [ ]:
fin_news_healines_dataset_pandas = financial_dataset.to_pandas()
label = []
for index,res in enumerate(fin_news_healines_dataset_pandas['sentiment_label']):
  if res.lower() == 'positive':
    label.append(2)
  elif res.lower() == 'negative':
    label.append(0)
  elif res.lower() == 'neutral':
    label.append(1)

In [ ]:
len(label)

2793

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# tokenizer.to(device)

Add **prompt** column to dataset

In [ ]:
labels = "positive/negative/neutral"
p=[]
for i in range(len(financial_dataset['Headlines'])):
  p.append(f"Predict the sentiment of the input. Answer either {labels}. Input: {financial_dataset['Headlines'][i]}. Answer:")

financial_dataset = financial_dataset.add_column("prompt_2", p)

In [ ]:
financial_dataset

Dataset({
    features: ['Headlines', 'sentiment_label', '__index_level_0__', 'prompt_2'],
    num_rows: 2793
})

In [ ]:
financial_dataset['prompt_2'][:3]

['Predict the sentiment of the input. Answer either positive/negative/neutral. Input: Petrol and Diesel Rate Today, 5 September: Some cities see revision; Check rates in Delhi, Mumbai, other cities. Answer:',
 'Predict the sentiment of the input. Answer either positive/negative/neutral. Input: The tightening of crude supply leads to a rise in oil prices.. Answer:',
 'Predict the sentiment of the input. Answer either positive/negative/neutral. Input: Nifty shows scope for downsides in near term, Buy these stocks to pocket gains. Answer:']

In [ ]:
# tokenized_set_1 = flant5_tokenizer(prompt, return_tensors='pt', padding=True,max_length=512)

#---------------------------------------------------------

tokenized_set_headlines = Internlm_quanto_tokenizer.batch_encode_plus(
    financial_dataset['prompt_2'], #prompt
    return_tensors='pt',
    add_special_tokens=True,
    return_attention_mask=True,
    # pad_to_max_length=True,  Deprecated
    padding=True,
    truncation=True,
    max_length=175 # the maximum lenght observed in the headlines, as can be seen in the above distribution
)

In [ ]:
input_ids_val = tokenized_set_headlines['input_ids']
attention_masks_val = tokenized_set_headlines['attention_mask']

dataset_val = TensorDataset(input_ids_val, attention_masks_val)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=5)

In [ ]:
import time, torch
predictions = []
start_time = time.time()
Internlm_quanto_model.eval()

for batch in dataloader_validation:

  batch = tuple(b.to(device) for b in batch)

  inputs = {'input_ids':batch[0],
                  'attention_mask': batch[1],
                 }

  with torch.no_grad():
    outputs = Internlm_quanto_model.generate(**inputs, max_length=200)
    # outputs = flant5_model(**inputs)

  res_sentences = [Internlm_quanto_tokenizer.decode(i) for i in outputs]
  # out_text = [o.split("Answer:")[1] for o in res_sentences]

  # # show results
  # for gen_text in out_text:
  #     print(gen_text)

  end_time = time.time()
  execution_time = end_time - start_time

  predictions.append(res_sentences)
# print("Result:", res_sentences)
print("Execution time:", execution_time, "seconds")

Execution time: 1882.7208585739136 seconds


**Extract response from models output**

In [ ]:
final_sent_answer=[]
for batch in predictions:
  for sent_output in batch:
    final_sent_answer.append(sent_output.split('Answer:')[1].split('<')[0])

In [ ]:
for index,res in enumerate(final_sent_answer):
  final_sent_answer[index] = res.replace(" ","")

In [ ]:
len(final_sent_answer)



2793

**Remove extra characters from output response**

In [ ]:
final_res_wo_extra_char=[]
def remove_extra_characters(string):

  # Define a set of extra characters to remove.
  extra_characters = set("!@#$%^&*()_+=`~.")

  # Create a new string without the extra characters.
  new_string = "".join([char for char in string if char not in extra_characters])

  # Return the new string.
  return new_string

for s in final_sent_answer:
  final_res_wo_extra_char.append(remove_extra_characters(s))

In [ ]:
final_res_wo_extra_char[:4]

['neutral', 'neutral', 'Positive', 'neutral']

In [ ]:
# prompt: convert list to dataframe with column called 'response'

import pandas as pd
df_res_headlines = pd.DataFrame(final_res_wo_extra_char, columns=['response'])


In [ ]:
df_res_headlines['label'] = [1] *2793

In [ ]:
df_res_headlines.head(5)

,response,label
0,neutral,1
1,neutral,1
2,Positive,2
3,neutral,1
4,Positive,2


In [ ]:
for index,res in enumerate(df_res_headlines['response']):
  # print(index)
  if (df_res_headlines['response'][index]).lower() == 'positive':
    df_res_headlines['label'][index] = 2
  elif (df_res_headlines['response'][index]).lower() == 'negative':
    df_res_headlines['label'][index] = 0
  elif (df_res_headlines['response'][index]).lower() == 'neutral':
    df_res_headlines['label'][index] = 1


In [ ]:
import pandas as pd
df_res_headlines.to_csv('final_sent_answer_Internlm-7b-quanto-balanced.csv')

Add **label** column to dataset

In [ ]:
fin_news_healines_dataset_pandas = financial_dataset.to_pandas()
label = []
for index,res in enumerate(fin_news_healines_dataset_pandas['sentiment_label']):
  if res.lower() == 'positive':
    label.append(2)
  elif res.lower() == 'negative':
    label.append(0)
  elif res.lower() == 'neutral':
    label.append(1)

In [ ]:
len(label)

2793

In [ ]:
# prompt: add new column to hugging face dataset object

financial_dataset = financial_dataset.add_column("label", label)


## Evaluate model

In [ ]:
financial_dataset

Dataset({
    features: ['Headlines', 'sentiment_label', '__index_level_0__', 'prompt_2', 'label'],
    num_rows: 2793
})

In [ ]:
len(label)

2793

In [ ]:
# prompt: code to convert dataset object fpb_dataset to pandas dataframe object

fin_news_healines_dataset_pandas = financial_dataset.to_pandas()
# Compare the values in the two columns
comparison = fin_news_healines_dataset_pandas["label"] == df_res_headlines["label"]

# Calculate the accuracy
accuracy = comparison.sum() / len(comparison)

# Print the accuracy
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.55


In [ ]:
# prompt: code to calculate precision

from sklearn.metrics import precision_score

y_true = fin_news_healines_dataset_pandas["label"]
y_pred = df_res_headlines["label"]

precision = precision_score(y_true, y_pred, average='macro')
print("Macro-averaged precision:", precision)

Macro-averaged precision: 0.5837228123444981


In [ ]:
# prompt: code to calculate f1 score

from sklearn.metrics import f1_score

# Calculate F1 score
f1 = f1_score(y_true, y_pred, average='macro')

# Print F1 score
print("Macro-averaged F1 score:", f1)


Macro-averaged F1 score: 0.5205582564130206


In [ ]:
# prompt: code to calculate recall

from sklearn.metrics import recall_score

# Calculate recall score
recall = recall_score(y_true, y_pred, average='macro')

# Print recall score
print("Macro-averaged recall:", recall)


Macro-averaged recall: 0.5501692977608639
